**1. Carga do Dataset**

In [ ]:
# Instalar o pacote Tidyverse
install.packages("tidyverse")
library(tidyverse)
library(ggplot2)


In [ ]:
# Carregar a biblioteca
library(readr)

# Carregar o arquivo CSV usando read_delim para arquivos delimitados por ponto e vírgula
dataset_eleitorado <- read_delim("perfil_eleitor_secao_ATUAL_RO.csv", delim = ";", locale = locale(encoding = "latin1"))

# Calcular dimensões do dataset
linhas <- nrow(dataset_eleitorado)
colunas <- ncol(dataset_eleitorado)
celulas <- linhas * colunas

cat("Número de linhas:", linhas, "\n")
cat("Número de colunas:", colunas, "\n")
cat("Número de células:", celulas, "\n")


**2. Seleção de Colunas (Transformação - Projeção)**

In [ ]:
# Selecionar apenas as colunas de interesse
colunas_interesse <- c("NM_MUNICIPIO", "CD_GENERO", "DS_GENERO", "CD_ESTADO_CIVIL",
                       "DS_ESTADO_CIVIL", "CD_FAIXA_ETARIA", "DS_FAIXA_ETARIA",
                       "CD_GRAU_ESCOLARIDADE", "DS_GRAU_ESCOLARIDADE", "QT_ELEITORES_PERFIL")

dataset_filtrado <- dataset_eleitorado %>% select(all_of(colunas_interesse))

# Comparar dimensões antes e depois
linhas_filtrado <- nrow(dataset_filtrado)
colunas_filtrado <- ncol(dataset_filtrado)
celulas_filtrado <- linhas_filtrado * colunas_filtrado

cat("Após projeção:\n")
cat("Linhas:", linhas_filtrado, "\n")
cat("Colunas:", colunas_filtrado, "\n")
cat("Células:", celulas_filtrado, "\n")

**3. Filtragem de Linhas (Transformação - Limpeza de Dados)**

In [ ]:
# Filtrar linhas inválidas
dataset_limpo <- dataset_filtrado %>%
  filter(
    DS_FAIXA_ETARIA != "Inválido",
    !str_detect(DS_GENERO, "NÃO INFORMADO"),
    !str_detect(DS_ESTADO_CIVIL, "NÃO INFORMADO"),
    !str_detect(DS_GRAU_ESCOLARIDADE, "NÃO INFORMADO")
  )

# Comparar dimensões antes e depois da limpeza
linhas_limpo <- nrow(dataset_limpo)
colunas_limpo <- ncol(dataset_limpo)
celulas_limpo <- linhas_limpo * colunas_limpo

cat("Após limpeza:\n")
cat("Linhas:", linhas_limpo, "\n")
cat("Colunas:", colunas_limpo, "\n")
cat("Células:", celulas_limpo, "\n")

2.3. Calcular o Número de Células com Dados

In [ ]:
# Contar o número de células com dados (ANTES E DEPOIS)
celulas_antes <- nrow(dataset_eleitorado) * ncol(dataset_eleitorado)
celulas_depois <- nrow(dataset_limpo) * ncol(dataset_limpo)

cat("Número de células antes das transformações:", celulas_antes, "\n")
cat("Número de células após as transformações:", celulas_depois, "\n")

Análises:

In [ ]:
# Filtrar dados com os critérios
filtro <- dataset_limpo %>%
  filter(
    DS_GENERO == "MASCULINO",
    DS_FAIXA_ETARIA == "35 a 39 anos",
    DS_ESTADO_CIVIL == "CASADO",
    DS_GRAU_ESCOLARIDADE == "ENSINO MÉDIO COMPLETO"
  )

# Calcular total de eleitores que atendem ao critério
total_filtro <- sum(filtro$QT_ELEITORES_PERFIL)

# Calcular total geral de eleitores
total_geral <- sum(dataset_limpo$QT_ELEITORES_PERFIL)

# Comparar
cat("Total de eleitores que atendem ao critério:", total_filtro, "\n")
cat("Total geral de eleitores:", total_geral, "\n")
cat("Proporção:", round((total_filtro / total_geral) * 100, 2), "%\n")


In [ ]:
# Contagem por sexo
contagem_sexo <- dataset_limpo %>%
  group_by(DS_GENERO) %>%
  summarise(QT_ELEITORES = sum(QT_ELEITORES_PERFIL))

# Contagem por município
contagem_municipio <- dataset_limpo %>%
  group_by(NM_MUNICIPIO) %>%
  summarise(QT_ELEITORES = sum(QT_ELEITORES_PERFIL))

# Combinação: Sexo versus Faixa Etária
sexo_vs_idade <- dataset_limpo %>%
  group_by(DS_GENERO, DS_FAIXA_ETARIA) %>%
  summarise(QT_ELEITORES = sum(QT_ELEITORES_PERFIL)) %>%
  arrange(desc(QT_ELEITORES))

print("Contagem por sexo")
print(contagem_sexo)
print("Contagem por município")
print(contagem_municipio)
print("Sexo vs Faixa Etária")
print(sexo_vs_idade)

In [ ]:
# Estatísticas básicas
estatisticas <- dataset_limpo %>%
  group_by(NM_MUNICIPIO) %>%
  summarise(
    Total_Eleitores = sum(QT_ELEITORES_PERFIL)
  ) %>%
  summarise(
    Media = mean(Total_Eleitores),
    Mediana = median(Total_Eleitores),
    Moda = Total_Eleitores[which.max(tabulate(match(Total_Eleitores, unique(Total_Eleitores))))],
    Minimo = min(Total_Eleitores),
    Maximo = max(Total_Eleitores),
    Intervalo = max(Total_Eleitores) - min(Total_Eleitores),
    Desvio_Padrao = sd(Total_Eleitores)
  )

print(estatisticas)


In [ ]:
# Comparação entre dois municípios
municipios_comparados <- dataset_limpo %>%
  filter(NM_MUNICIPIO %in% c("Município A", "Município B")) %>%
  group_by(NM_MUNICIPIO, DS_GENERO) %>%
  summarise(Total_Eleitores = sum(QT_ELEITORES_PERFIL))

print(municipios_comparados)


In [ ]:
# Total de eleitores agrupados por município e grau de instrução
agrupado_municipio <- dataset_limpo %>%
  group_by(NM_MUNICIPIO, DS_GRAU_ESCOLARIDADE) %>%
  summarise(QT_ELEITORES = sum(QT_ELEITORES_PERFIL))

print(agrupado_municipio)


In [ ]:
# Município com menos eleitores
municipio_menos <- dataset_limpo %>%
  group_by(NM_MUNICIPIO) %>%
  summarise(Total_Eleitores = sum(QT_ELEITORES_PERFIL)) %>%
  arrange(Total_Eleitores) %>%
  slice(1)

# Município com mais eleitores (exceto a capital)
municipio_mais <- dataset_limpo %>%
  filter(NM_MUNICIPIO != "Capital") %>%
  group_by(NM_MUNICIPIO) %>%
  summarise(Total_Eleitores = sum(QT_ELEITORES_PERFIL)) %>%
  arrange(desc(Total_Eleitores)) %>%
  slice(1)

cat("Município com menos eleitores:\n")
print(municipio_menos)

cat("Município com mais eleitores (exceto a capital):\n")
print(municipio_mais)


In [ ]:

ggplot(contagem_sexo, aes(x = DS_GENERO, y = QT_ELEITORES, fill = DS_GENERO)) +
  geom_bar(stat = "identity") +
  labs(title = "Distribuição de Eleitores por Gênero",
       x = "Gênero", y = "Total de Eleitores") +
  theme_minimal()


In [ ]:
ggplot(sexo_vs_idade, aes(x = DS_FAIXA_ETARIA, y = QT_ELEITORES, fill = DS_GENERO)) +
  geom_bar(stat = "identity", position = "dodge") +
  labs(title = "Eleitores por Faixa Etária e Sexo",
       x = "Faixa Etária", y = "Total de Eleitores") +
  theme_minimal()


In [ ]:
# Heatmap Faixa Etária vs Estado Civil para Homens
heatmap_homens <- dataset_limpo %>%
  filter(DS_GENERO == "MASCULINO") %>%
  group_by(DS_FAIXA_ETARIA, DS_ESTADO_CIVIL) %>%
  summarise(QT_ELEITORES = sum(QT_ELEITORES_PERFIL))

ggplot(heatmap_homens, aes(x = DS_FAIXA_ETARIA, y = DS_ESTADO_CIVIL, fill = QT_ELEITORES)) +
  geom_tile() +
  scale_fill_gradient(low = "white", high = "blue") +
  labs(title = "Heatmap: Faixa Etária x Estado Civil (Homens)",
       x = "Faixa Etária", y = "Estado Civil") +
  theme_minimal()


In [ ]:
# Heatmap para Faixa Etária vs Estado Civil - Gênero Feminino
heatmap_feminino <- dataset_limpo %>%
  filter(DS_GENERO == "FEMININO") %>%
  group_by(DS_FAIXA_ETARIA, DS_ESTADO_CIVIL) %>%
  summarise(QT_ELEITORES = sum(QT_ELEITORES_PERFIL), .groups = "drop") %>%
  ggplot(aes(x = DS_ESTADO_CIVIL, y = DS_FAIXA_ETARIA, fill = QT_ELEITORES)) +
  geom_tile(color = "white") +
  scale_fill_gradient(low = "green", high = "red", name = "Eleitores") +
  labs(
    title = "Distribuição de Eleitores (Feminino)",
    x = "Estado Civil",
    y = "Faixa Etária"
  ) +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))